# 닥터풋헬스 스마트스토어 고객 리뷰 수집
- 링크: https://smartstore.naver.com/foothealth/products/4868868795
- 수집할 데이터 요소: 
    1. 평점 (Rating): `rating`
    2. 아이디 (User ID): `user_id`
    3. 작성날짜 (Review Date): `review_date`
    4. 제품명 (Product Name): `product_name`
    5. 간단리뷰 (Brief Review): `brief_review`
    6. 리뷰유형 (Review Type): `review_type`
        - 이 변수는 '한달사용기', '재구매', 'BEST' 등의 값을 포함할 수 있다.
    7. 리뷰내용 (Review Content): `review_content`
    8. 추천수 (Number of Likes/Recommendations): `recommendation_count`

In [1]:
# -*- coding: utf-8 -*-
from datetime import datetime
import pandas as pd
import numpy as np
import time, random
import re
from tqdm.notebook import trange, tqdm

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [2]:
# Chrome Webdriver 불러오기
def chromeWebdriver():
    # 크롬 드라이버 자동 설치
    chrome_service = ChromeService(executable_path=ChromeDriverManager().install())
    
    # 드라이버 옵션 설정
    options = Options()
    options.add_experimental_option('detach', True) # 브라우저 바로 닫힘 방지
    options.add_experimental_option('excludeSwitches', ['enable-logging']) # 불필요한 메시지 제거
    options.add_argument('--headless') # 백그라운드 작업
    options.add_argument('--no-sandbox') # Bypass OS security model

    driver = webdriver.Chrome(service=chrome_service, options=options)
    
    return driver

In [5]:
# 스크래핑할 스마트스토어 주소 불러오기
url = "https://smartstore.naver.com/nineto6/products/3989586411"
driver = chromeWebdriver()
driver.get(url)

SCROLL_PAUSE_SEC = 1

# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 높이 초기화
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height: # 새로 업로드되는 페이지가 없다면
        break
    last_height = new_height
    
# 리뷰 버튼 클릭
driver.find_element(By.XPATH, r'//*[@id="_productFloatingTab"]/div/div[3]/ul/li[2]').click()
time.sleep(1)
# 최신순 정렬 클릭
driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[2]').click()
time.sleep(1)

# 리뷰 개수로 페이지 수 계산
num_reviews = driver.find_element(By.XPATH, r'//*[@id="content"]/div/div[2]/div[1]/div[2]/div[1]/a/strong').text 
num_reviews = int(num_reviews.replace(",", ""))
max_page = int(num_reviews//20 + 1)
print('리뷰 수: ',num_reviews,', 페이지 수: ', max_page)

리뷰 수:  692 , 페이지 수:  35


In [6]:
# 각 크롤링 결과 저장하기 위한 리스트 선언 
rating = []
user_id = []
review_date = []
product_info = []
review_text = []
recommendation_count = []
image_yn = []

# 페이지를 처음부터 끝까지, (20년도까지) 이동하면서 데이터 수집
page = 1
date = '2024.01.01'

while (page <= max_page) & ('20.01.01' <= date) :
    lis = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul').find_elements(By.TAG_NAME, 'li') 
    print('● 현재 페이지:', page, ', 해당 페이지 리뷰수:', len(lis))

    for i in range(1, len(lis) + 1):
        try: 
            rating.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[1]/em').text)
            driver.implicitly_wait(1)
        except:
            rating.append(np.nan)
        try:    
            user_id.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/strong').text)
            driver.implicitly_wait(1)
        except:
            user_id.append(np.nan)
        try: 
            date = driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[2]/span').text
            review_date.append(date)
            driver.implicitly_wait(1)
        except:
            review_date.append(np.nan)
        try:
            product_info.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[1]/div[2]/div[3]').text)
            driver.implicitly_wait(1)
        except:
            product_info.append(np.nan)
        try: 
            review_text.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[1]/div[2]/div').text)
            driver.implicitly_wait(1)
        except:
            review_text.append(np.nan)
        try: 
            recommendation_count.append(driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[2]/div/div/div/button/span').text)
            driver.implicitly_wait(1)
        except:
            recommendation_count.append(np.nan)
        try:
            driver.find_element(By.XPATH, r'//*[@id="REVIEW"]/div/div[3]/div[2]/ul/li['+str(i)+']/div/div/div/div[1]/div/div[2]')
            image_yn.append('Y')
        except:
            image_yn.append('N')
            
    # click next button
    driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()
    time.sleep(1)
    page += 1

driver.quit()
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

● 현재 페이지: 1 , 해당 페이지 리뷰수: 20
● 현재 페이지: 2 , 해당 페이지 리뷰수: 20
● 현재 페이지: 3 , 해당 페이지 리뷰수: 20
● 현재 페이지: 4 , 해당 페이지 리뷰수: 20
● 현재 페이지: 5 , 해당 페이지 리뷰수: 20
● 현재 페이지: 6 , 해당 페이지 리뷰수: 20
● 현재 페이지: 7 , 해당 페이지 리뷰수: 20
● 현재 페이지: 8 , 해당 페이지 리뷰수: 20
● 현재 페이지: 9 , 해당 페이지 리뷰수: 20
● 현재 페이지: 10 , 해당 페이지 리뷰수: 20
● 현재 페이지: 11 , 해당 페이지 리뷰수: 20
● 현재 페이지: 12 , 해당 페이지 리뷰수: 20
● 현재 페이지: 13 , 해당 페이지 리뷰수: 20
● 현재 페이지: 14 , 해당 페이지 리뷰수: 20
● 현재 페이지: 15 , 해당 페이지 리뷰수: 20
● 현재 페이지: 16 , 해당 페이지 리뷰수: 20
● 현재 페이지: 17 , 해당 페이지 리뷰수: 20
● 현재 페이지: 18 , 해당 페이지 리뷰수: 20
● 현재 페이지: 19 , 해당 페이지 리뷰수: 20
● 현재 페이지: 20 , 해당 페이지 리뷰수: 20
● 현재 페이지: 21 , 해당 페이지 리뷰수: 20
● 현재 페이지: 22 , 해당 페이지 리뷰수: 20
● 현재 페이지: 23 , 해당 페이지 리뷰수: 20
● 현재 페이지: 24 , 해당 페이지 리뷰수: 20
● 현재 페이지: 25 , 해당 페이지 리뷰수: 20
● 현재 페이지: 26 , 해당 페이지 리뷰수: 20


,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.02.,hanj********,5,두족 구매시 두번 선택해주세요: 블랙270~275mm\n마감처리깔끔해요사이즈정사이즈예요,한달사용기아주 좋은 제품입니다,N,0
1,24.01.01.,oasi********,5,두족 구매시 두번 선택해주세요: 블랙250~255mm\n마감처리깔끔해요사이즈정사이즈예요,문제없이 잘왔습니다.,N,0
2,23.12.26.,as****,5,두족 구매시 두번 선택해주세요: 블랙230~235mm\n마감처리아주 깔끔해요사이즈정...,한달사용기재구매편해서 아이꺼에도 사줬는데 만족합니다.,Y,0
3,23.12.26.,as****,5,두족 구매시 두번 선택해주세요: 블랙260~265mm\n마감처리아주 깔끔해요사이즈정...,한달사용기재구매낄창 편하고 좋아요.\n족저근막염 있는분들에게 추천이요,Y,0
4,23.12.26.,tjsg******,5,두족 구매시 두번 선택해주세요: 블랙260~265mm\n평소사이즈265mm마감처리아...,한달사용기하루에 2만보이상 걸어도 발의 피로가 확 줄었어요.,Y,0


In [6]:
df = pd.DataFrame(list(zip(review_date, user_id, rating, product_info, review_text, image_yn, recommendation_count)), columns = ['review_date', 'user_id', 'rating', 'product_info', 'review_text', 'image_yn', 'recommendation_count'])
df.head()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.02.,so****,5,인솔: 3FEET HIGH / 사이즈: Medium (250~265mm)\n마감처리...,상품평 보고 처음부터 High로 골라서 \n잘 적응하고 있습니다,Y,0
1,24.01.02.,upsp***,5,인솔: 3FEET MID / 사이즈: Small (235~245mm)\n마감처리깔끔...,재구매배송빠르고 아들발에 이것만써요,N,0
2,23.12.31.,xorm********,5,인솔: 3FEET HIGH / 사이즈: Medium (250~265mm)\n평소사이...,아직 큰 차이는 모르겠지만 걸을때 더 편해진 것 같아요,N,0
3,23.12.30.,yunm****,5,인솔: 3FEET HIGH / 사이즈: Medium (250~265mm)\n평소사이...,사용하니 발이펀안하고 안아파졌어요~좋아요,N,0
4,23.12.30.,knps***,5,인솔: 3FEET MID / 사이즈: Medium (250~265mm)\n마감처리깔...,우선 아치가 낮아서 오래걸으면 피곤해하고. 아치가 낮아서 중심이 무너진다해서 병...,Y,0


In [7]:
df.to_excel("data/WalkerMaster.xlsx", index=False)

In [9]:
df = pd.read_excel('data/FootMedical.xlsx')
df.head()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
0,24.01.02.,gg****,5,교정구선택: 원스텝(회색)전사이즈 / 신발사이즈: 265mm(8호)\n마감처리깔끔해...,한달사용기좋아요 굿굿굿!!!,N,0
1,24.01.01.,wang***,5,교정구선택: 원스텝(회색)전사이즈 / 신발사이즈: 250mm(6호)\n마감처리깔끔해...,회색이 조금더 딱딱합니다\n신랑은 흰색 전회색\n전 전에도 썼고 신랑은 처음\n처음...,Y,0
2,24.01.01.,wang***,5,교정구선택: 사이언스(크림) / 신발사이즈: 270mm(A8호)\n평소사이즈245m...,전에 쓰던걸 신발버리면서 버려서 비슷하길래 구매해봤어요ㅎ 생각보다 좋아요\n전에도 ...,Y,0
3,23.12.30.,suks*******,5,교정구선택: 원스텝(회색)전사이즈 / 신발사이즈: 265mm(8호)\n마감처리깔끔해...,남편이 평발인데 걷기운동하고 와서 편하고 좋다고 하네요 저도 같이 구매했는데 좀 더...,Y,0
4,23.12.30.,suks*******,5,교정구선택: 원스텝(회색)전사이즈 / 신발사이즈: 240mm(5호)\n마감처리깔끔해...,잘받았습니다 일단 신어보니 편한것 같은데 아직은 잘 모르겠어요,Y,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7760 entries, 0 to 7759
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   review_date           7760 non-null   object
 1   user_id               7760 non-null   object
 2   rating                7760 non-null   int64 
 3   product_info          7760 non-null   object
 4   review_text           7760 non-null   object
 5   image_yn              7760 non-null   object
 6   recommendation_count  7760 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 424.5+ KB


In [11]:
df.tail()

,review_date,user_id,rating,product_info,review_text,image_yn,recommendation_count
7755,19.12.22.,tnal*****,5,신발사이즈: 250mm(6호)\n마감처리깔끔해요사이즈정사이즈예요,단단하고 너무 좋습니다 고정이 되니 발이 편해요,N,0
7756,19.12.21.,scar*******,5,신발사이즈: 245mm(6호)\n마감처리아주 깔끔해요사이즈생각보다 커요,아직 잘 모르겠지만 나름 만족합니다~,Y,0
7757,19.12.21.,pate*****,4,신발사이즈: 275mm(9호)\n마감처리깔끔해요사이즈정사이즈예요,"나쁘지 않습니다.\n아치부분을 지압하는것인데, 좋아요",N,0
7758,19.12.20.,haa0***,5,신발사이즈: 240mm(5호)\n마감처리깔끔해요사이즈정사이즈예요,편하고 아주 좋아요 \n더 신어보고 올릴께요@,N,0
7759,19.12.20.,an****,5,신발사이즈: 230mm(4호)\n마감처리깔끔해요사이즈정사이즈예요,아직 쓰기전인데 한번 느껴보겠습니다,N,0
